<a href="https://colab.research.google.com/github/BirukovAlex/neto_Python/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BF%D0%BE_%D1%82%D0%B5%D0%BC%D0%B5_%C2%AB%D0%A0%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%86%D0%B8%D0%B8_%D0%BD%D0%B0_%D0%BE%D1%81%D0%BD%D0%BE%D0%B2%D0%B5_%D1%81%D0%BE%D0%B4%D0%B5%D1%80%D0%B6%D0%B0%D0%BD%D0%B8%D1%8F%C2%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Использовать датасет https://files.grouplens.org/datasets/movielens/ml-latest.zip
2. Построить рекомендации (регрессия, предсказываем оценку) на фичах:

-TF-IDF на тегах и жанрах;

-средние оценки (+ median, variance и т. д.) пользователя и фильма.

3. Оценить RMSE на тестовой выборке

#Загрузка библиотек и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [2]:
# Для загрузки с помощью python можно использовать библиотеки zipfile, requests, os
# Мы загрузили дата сеты по ссылке вручную (минимальную базу).
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

#Подготовка фич

In [3]:
# Создаем фичи для фильмов

def prepare_movie_features():
    # TF-IDF на жанрах
    movies['genres'] = movies['genres'].str.replace('|', ' ')

    # TF-IDF на тегах
    movie_tags = tags.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)).reset_index()
    movie_tags.columns = ['movieId', 'tags']

    # Объединяем жанры и теги
    movie_features = movies.merge(movie_tags, on='movieId', how='left')
    movie_features['tags'] = movie_features['tags'].fillna('')
    movie_features['text_features'] = movie_features['genres'] + ' ' + movie_features['tags']

    return movie_features

# Создаем статистики по пользователям и фильмам
def create_statistical_features(ratings_df):
    # Статистики по пользователям
    user_stats = ratings_df.groupby('userId')['rating'].agg([
        'mean', 'median', 'std', 'count', 'var'
    ]).reset_index()
    user_stats.columns = ['userId', 'user_mean', 'user_median', 'user_std', 'user_count', 'user_var']

    # Статистики по фильмам
    movie_stats = ratings_df.groupby('movieId')['rating'].agg([
        'mean', 'median', 'std', 'count', 'var'
    ]).reset_index()
    movie_stats.columns = ['movieId', 'movie_mean', 'movie_median', 'movie_std', 'movie_count', 'movie_var']

    return user_stats, movie_stats

# Подготовка данных
print("Подготавливаем фичи...")
movie_features = prepare_movie_features()
user_stats, movie_stats = create_statistical_features(ratings)

# Объединяем все данные
data = ratings.merge(user_stats, on='userId')
data = data.merge(movie_stats, on='movieId')
data = data.merge(movie_features[['movieId', 'text_features']], on='movieId')

Подготавливаем фичи...


#TF-IDF преобразование

In [4]:
# Создаем TF-IDF фичи
print("Создаем TF-IDF фичи...")
tfidf = TfidfVectorizer(max_features=1000, stop_words='english')
tfidf_features = tfidf.fit_transform(data['text_features'])

# Преобразуем в DataFrame
tfidf_df = pd.DataFrame(tfidf_features.toarray(),
                       columns=[f'tfidf_{i}' for i in range(tfidf_features.shape[1])])
tfidf_df.index = data.index

# Объединяем все фичи
feature_columns = ['user_mean', 'user_median', 'user_std', 'user_count', 'user_var',
                  'movie_mean', 'movie_median', 'movie_std', 'movie_count', 'movie_var']

X = pd.concat([data[feature_columns], tfidf_df], axis=1)
y = data['rating']

# Заполняем пропущенные значения
X = X.fillna(0)

Создаем TF-IDF фичи...


#Разделение на train/test, обучение моделей и сравнение RMSE

In [7]:
# Разделяем данные
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Размер train: {X_train.shape}, test: {X_test.shape}")

# 1. Базовые модели для сравнения
print("\n1. Базовые модели для сравнения:")

# Только средние оценки (baseline)
baseline_pred = np.full_like(y_test, y_train.mean())
baseline_rmse = np.sqrt(mean_squared_error(y_test, baseline_pred))
print(f"Baseline (всегда средняя оценка): {baseline_rmse:.4f}")

# 2. Только статистические фичи (без TF-IDF)
print("\n2. Только статистические фичи пользователя/фильма:")
statistical_features = ['user_mean', 'user_median', 'user_std', 'user_count', 'user_var',
                       'movie_mean', 'movie_median', 'movie_std', 'movie_count', 'movie_var']

X_train_stats = X_train[statistical_features]
X_test_stats = X_test[statistical_features]

# Linear Regression на статистических фичах
lr_stats = LinearRegression()
lr_stats.fit(X_train_stats, y_train)
lr_stats_pred = lr_stats.predict(X_test_stats)
lr_stats_rmse = np.sqrt(mean_squared_error(y_test, lr_stats_pred))
print(f"Linear Regression (только статистики): {lr_stats_rmse:.4f}")

# Random Forest на статистических фичах
rf_stats = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_stats.fit(X_train_stats, y_train)
rf_stats_pred = rf_stats.predict(X_test_stats)
rf_stats_rmse = np.sqrt(mean_squared_error(y_test, rf_stats_pred))
print(f"Random Forest (только статистики): {rf_stats_rmse:.4f}")

# 3. Полная модель со всеми фичами (включая TF-IDF)
print("\n3. Полная модель со всеми фичами (включая TF-IDF):")

# Линейная регрессия
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)
lr_rmse = np.sqrt(mean_squared_error(y_test, lr_pred))
print(f"Linear Regression (все фичи): {lr_rmse:.4f}")

# Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))
print(f"Random Forest (все фичи): {rf_rmse:.4f}")

# 4. Анализ улучшений
print("\n4. Анализ улучшений от TF-IDF:")
print(f"Улучшение Linear Regression: {lr_stats_rmse - lr_rmse:.4f} ({((lr_stats_rmse - lr_rmse)/lr_stats_rmse*100):.1f}%)")
print(f"Улучшение Random Forest: {rf_stats_rmse - rf_rmse:.4f} ({((rf_stats_rmse - rf_rmse)/rf_stats_rmse*100):.1f}%)")

Размер train: (80668, 1010), test: (20168, 1010)

1. Базовые модели для сравнения:
Baseline (всегда средняя оценка): 1.0488

2. Только статистические фичи пользователя/фильма:
Linear Regression (только статистики): 0.8112
Random Forest (только статистики): 0.8157

3. Полная модель со всеми фичами (включая TF-IDF):
Linear Regression (все фичи): 0.8166
Random Forest (все фичи): 0.8087

4. Анализ улучшений от TF-IDF:
Улучшение Linear Regression: -0.0054 (-0.7%)
Улучшение Random Forest: 0.0071 (0.9%)
